In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pickle

DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

DATA_FILE = 'data/data.pkl'
with open(DATA_FILE, 'rb') as f:
    Cs = pickle.load(f)
    
DATA_FILE2 = 'data/labels.pkl'  
with open(DATA_FILE2, 'rb') as f:
    Ys = pickle.load(f)

In [5]:
print(Cs[0][0].entities[0])

EntityMention(doc_id=1: 'bill'(6:10)


In [ ]:
print(neu_adjectives_list)

In [ ]:
import collections

collections.Counter(Ys[0])

In [ ]:
print(f"Train Size: {len(Cs[0])}")
print(f"Dev Size:   {len(Cs[1])}")
print(f"Test Size:  {len(Cs[2])}")
print(f"Train Size: {len(Ys[0])}")
print(f"Dev Size:   {len(Ys[1])}")
print(f"Test Size:  {len(Ys[2])}")

In [18]:
from babble.utils import ExplanationIO2

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO2()
explanations = exp_io.read(FILE)

Read 377 explanations from data/my_explanations.tsv


In [20]:
print(explanations[0].condition)

the word "total" is within 2 words of "bill"


In [19]:
from babble import Babbler
babbler = Babbler(Cs, Ys,apply_filters=False)

Grammar construction complete.


In [ ]:
babbler.add_aliases({'positive': pos_adjectives_list})
babbler.add_aliases({'negative': neg_adjectives_list})
babbler.add_aliases({'neutral': neu_adjectives_list})

In [20]:
babbler.apply(explanations, split=0)

- | #                                                 | 2 Elapsed Time: 0:00:00

Building list of target candidate ids...
Collected 0 unique target candidate ids from 377 explanations.
No candidate hashes were provided. Skipping linking.


| |         #                                       | 376 Elapsed Time: 0:00:10


377 explanation(s) out of 377 were parseable.
377 parse(s) generated from 377 explanation(s).
Because apply_filters=False, no parses are being filtered.
Applying labeling functions to investigate labeling signature.
[========================================] 100%

Added 377 parse(s) from 377 explanations to set. (Total # parses = 377)

Applying labeling functions to split 1
[========================================] 100%

Added 0 labels to split 1: L.nnz = 0, L.shape = (812, 377).
Applying labeling functions to split 2
[========================================] 100%

Added 0 labels to split 2: L.nnz = 0, L.shape = (812, 377).


In [15]:
Ls = []
for split in [0,1,2]:
    L = babbler.get_label_matrix(split)
    Ls.append(L)

AttributeError: 'NoneType' object has no attribute 'nnz'

In [ ]:
print(Ls[0])

In [84]:
parses = babbler.get_parses(translate=False)

In [85]:
semantics = [parse.semantics for parse in parses ]
for semantic in semantics:
    print(semantic)

('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'bill')), ('.string', 'total'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'service')), ('.string', 'horrible'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'place')), ('.string', 'awful'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'staff')), ('.string', 'awful'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'freezer')), ('.string', 'disappointed'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'cream')), ('.string', 'strawberry'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'hour')), ('.string', 'happy'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'food')), ('.string', 'great'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'place')), ('.string', 'worse'))))
('.root', ('.label', ('.int', 1), ('.call', ('.eq', ('.string', 'meal')), ('.string', 'unsati

In [88]:
print(parses[0].function("bill"))

0


In [ ]:
parses[1].function(Cs[0][0])

In [ ]:
parses[2].semantics

In [ ]:
from metal import LabelModel

label_aggregator = LabelModel(5)
label_aggregator.train(Ls[0], n_epochs=100, lr=0.01)
label_aggregator.score(Ls[1], Ys[1])

In [ ]:
Y_p = label_aggregator.predict(Ls[0])

In [ ]:
with open("Ls.pkl", 'wb') as f:
    pickle.dump(Ls, f)
    
with open("Y_p.pkl", 'wb') as f:
    pickle.dump(Y_p, f)
    
with open("data/wrong_indexes.pkl", 'wb') as f:
    pickle.dump(wrong_indexes, f)

In [ ]:
from src.PipelineReviewPos.result_analysis import analyze

analyze(Ls,parses)

In [ ]:
print()